In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer

In [86]:
def cleanData(X):
    X.Sex = X.Sex.replace(["female", "male"], [1, 0])
    
    X.Embarked = X.Embarked.replace(["C", "Q", "S", np.NaN], [3, 2, 1, 3])


In [87]:
rawData = pd.read_csv('titanic/train.csv')
#rawData.describe()
rawData.isnull().sum()
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = rawData[features]
cleanData(X)

y = rawData['Survived']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
decisionModel = DecisionTreeRegressor(random_state = 1)

imp = SimpleImputer(missing_values = -1, strategy='mean')
X_train = pd.DataFrame(imp.fit_transform(X_train))
X_train.columns = X_test.columns

X_train.head()
#y_train.head()
decisionModel.fit(X_train,y_train)
yPredicted = decisionModel.predict(X_test)
err = mean_absolute_error(yPredicted, y_test)
print(err)



0.24436711768550334


C:\Users\darin\AppData\Local\Temp\ipykernel_61004\522747421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Sex = X.Sex.replace(["female", "male"], [1, 0])
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\522747421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Embarked = X.Embarked.replace(["C", "Q", "S", np.NaN], [3, 2, 1, 3])


In [109]:
#Test
finalData = pd.read_csv('titanic/test.csv') #6873
X = finalData[features]

X.columns = X_test.columns
cleanData(X)
X.Fare = X.Fare.replace(np.nan, 1000)
X = pd.DataFrame(imp.transform(X))
ans = decisionModel.predict(X)
ans = [round(num) for num in ans]
output = pd.DataFrame({'Id': finalData.PassengerId, 'Survive': ans})
output.to_csv('ans.csv', index=False)

C:\Users\darin\AppData\Local\Temp\ipykernel_61004\522747421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Sex = X.Sex.replace(["female", "male"], [1, 0])
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\522747421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Embarked = X.Embarked.replace(["C", "Q", "S", np.NaN], [3, 2, 1, 3])
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\2340421335.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin